In [ ]:
!wget https://www.dropbox.com/s/nilt43hyl1dx82k/dataset.zip?dl=0



# 新段落

# 新段落

# 新段落

In [ ]:
!unzip dataset.zip?dl=0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras_preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.losses import categorical_crossentropy

In [ ]:
train_datagen = ImageDataGenerator(
     zoom_range = 0.2, 
     shear_range = 0.2, 
     horizontal_flip=True, 
     rescale = 1./255
)

train_data = train_datagen.flow_from_directory(directory= "/content/train", 
                                               target_size=(224,224), 
                                               batch_size=32,
                                  )

In [ ]:
train_data.class_indices

In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255 )

val_data = val_datagen.flow_from_directory(directory= "/content/test", 
                                           target_size=(224,224), 
                                           batch_size=32,
                                  )

In [ ]:
#mobilenet module

base_model = MobileNet( input_shape=(224,224,3), include_top= False )

for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=7 , activation='softmax' )(x)

# softmax should be proper... maybe?
model = Model(base_model.input, x)

In [ ]:
model.compile(optimizer='adam', loss= categorical_crossentropy , metrics=['accuracy']  )

In [ ]:
#showing some pictures
t_img , label = train_data.next()

def plotImages(img_arr, label):

  count = 0
  for im, l in zip(img_arr,label) :
    plt.imshow(im)
    
    plt.title({i for i in train_data.class_indices if train_data.class_indices[i]==np.nonzero(l)[0][0]})
    plt.axis = False
    plt.show()
    
    count += 1
    if count == 5:
      break

plotImages(t_img, label)

In [ ]:
# having early stopping and model check point 

from keras.callbacks import ModelCheckpoint, EarlyStopping

es = EarlyStopping(monitor='val_accuracy', min_delta= 0.01 , patience= 5, verbose= 1, mode='auto')

mc = ModelCheckpoint(filepath="best_model.h5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto')
call_back = [es, mc]

In [ ]:
hist = model.fit_generator(train_data, 
                           steps_per_epoch= 10, 
                           epochs= 30, 
                           validation_data= val_data, 
                           validation_steps= 8, 
                           callbacks=[es,mc])

In [ ]:
# Loading the best fit model 
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [ ]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

In [ ]:
h =  hist.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'] , c = "red")
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'] , c = "red")
plt.title("loss vs v-loss")
plt.show()

In [ ]:
# just to map o/p values 
op = dict(zip( train_data.class_indices.values(), train_data.class_indices.keys()))

In [ ]:
# path for the image to see if it predics correct class

path = "/content/test/angry/PrivateTest_1488292.jpg"
img = load_img(path, target_size=(224,224) )

i = img_to_array(img)/255
input_arr = np.array([i])
input_arr.shape

pred = np.argmax(model.predict(input_arr))

print(f" the image is of {op[pred]}")

# to display the image  
plt.imshow(input_arr[0])
plt.title("input image")
plt.show()

In [ ]:
pip install face-recognition


In [ ]:
import face_recognition
import cv2
import numpy as np

In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")
from keras_preprocessing.image import load_img, img_to_array 
from keras.models import  load_model
import matplotlib.pyplot as plt
import numpy as np

# load model
model = load_model("best_model.h5")


face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

path = "/content/test_hreso/angry/images.jpg"
img = load_img(path, target_size=(224,224) )

#cap = cv2.VideoCapture(0)


#ret, test_img = cap.read()  # captures frame and returns boolean value and captured image
#test_img = img
#gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

test_img = cv2.imread(path)	
gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)


faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
faces_detected

In [ ]:
import os
import cv2
import dlib
import numpy as np
from keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")
from keras_preprocessing.image import load_img, img_to_array 
from keras.models import  load_model
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
# load model
model = load_model("best_model.h5")

#maybe dlib detector is better?
#face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

path = "/content/test_hreso/angry/images.jpg"
#img = load_img(path, target_size=(224,224) )

#cap = cv2.VideoCapture(0)


#ret, test_img = cap.read()  # captures frame and returns boolean value and captured image
#test_img = img
#gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)


detector_lm = dlib.get_frontal_face_detector()
predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

test_img = cv2.imread(path)	
gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
cv2_imshow(gray_img)

faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
rects = detector_lm(gray_img, 1)
for rect in rects:
    x = rect.tl_corner().x
    y = rect.tl_corner().y
    w = rect.width()
    h = rect.height()
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
    roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
    roi_gray = cv2.resize(roi_gray, (224, 224))
    img_pixels = img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis=0)
    img_pixels /= 255

    predictions = model.predict(img_pixels)

      # find max indexed array
    max_index = np.argmax(predictions[0])

    emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    predicted_emotion = emotions[max_index]

    cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    shape = predictor_lm(gray_img, rect)
    shape_np = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    shape = shape_np

        # Display the landmarks
    for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
        cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)

#resized_img = cv2.resize(test_img, (400, 400))

#cv2_imshow(resized_img)



In [ ]:
# import the modules
import os
from os import listdir
 
# get the path/directory
count = 0

total = 0

correct = 0
TP_class = [0]*7
FP_class = [0]*7
FN_class = [0]*7

table = np.arange(49).reshape((7,7))
for emo in range(7):
  folder_dir = "/content/test_hreso/" + op[emo] + "/"
  print(folder_dir)
  for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):
        test_img = cv2.imread(folder_dir+images)
        #cv2_imshow(img)  
        gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
        #cv2_imshow(gray_img)    
        faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        if len(faces_detected) == 0:
          continue
        for (x, y, w, h) in faces_detected:
          cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
          roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
          roi_gray = cv2.resize(roi_gray, (224, 224))
          img_pixels = img_to_array(roi_gray)
          img_pixels = np.expand_dims(img_pixels, axis=0)
          img_pixels /= 255

          predictions = model.predict(img_pixels)

      # find max indexed array
          max_index = np.argmax(predictions[0])
          total += 1
          table[emo,max_index]+=1
          if max_index == emo:

            correct += 1
            TP_class[emo] += 1
          else:
            FP_class[max_index] += 1
            FN_class[emo] += 1
          emotions = ('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')
          predicted_emotion = emotions[max_index]

          cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        resized_img = cv2.resize(test_img, (400, 400))

        cv2_imshow(resized_img)
        
        #count += 1
        #if count > 5:
        #  count = 0
        #  break
prec_class = [0]*7
recl_class = [0]*7
for emo in range(7):
  trueP = table[emo,emo]
  falseN = -trueP
  for emo_i in range(7):
    falseN += table[emo_i,emo]
  falseP = -trueP
  for emo_i in range(7):
    falseP += table[emo,emo_i]
  print(trueP)
  print(trueP/falseN)
  print(trueP/falseP)



In [ ]:
prec_class = [0]*7
recl_class = [0]*7
for emo in range(7):
  trueP = table[emo,emo]
  falseN = -trueP
  for emo_i in range(7):
    falseN += table[emo_i,emo]
  falseP = -trueP
  for emo_i in range(7):
    falseP += table[emo,emo_i]
  print(trueP)
  prec_class[emo] = (trueP/(trueP+falseP))
  recl_class[emo] = (trueP/(trueP+falseN))
print(prec_class)
print(recl_class)

In [ ]:
# import the modules
import os
from os import listdir
 
# get the path/directory
count = 0

total = 0

correct = 0
TP_class = [0]*7
FP_class = [0]*7
FN_class = [0]*7
detector_lm = dlib.get_frontal_face_detector()
predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

table = np.arange(49).reshape((7,7))
for emo in range(7):
  folder_dir = "/content/test_hreso/" + op[emo] + "/"
  print(folder_dir)
  for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):
        test_img = cv2.imread(folder_dir+images)
        #cv2_imshow(img)  
        gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
        #cv2_imshow(gray_img)    
        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        #if len(faces_detected) == 0:
        #cv2_imshow(gray_img)

        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        rects = detector_lm(gray_img, 1)
        if len(rects) == 0:
          continue
        for rect in rects:
          x = rect.tl_corner().x
          y = rect.tl_corner().y
          w = rect.width()
          h = rect.height()
          cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
          roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
          if roi_gray.size == 0:
            continue
          roi_gray = cv2.resize(roi_gray, (224, 224))
          img_pixels = img_to_array(roi_gray)
          img_pixels = np.expand_dims(img_pixels, axis=0)
          img_pixels /= 255

          predictions = model.predict(img_pixels)

      # find max indexed array
          max_index = np.argmax(predictions[0])
          total += 1
          table[emo,max_index]+=1
          if max_index == emo:

            correct += 1
            TP_class[emo] += 1
          else:
            FP_class[max_index] += 1
            FN_class[emo] += 1
          emotions = ('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')
          predicted_emotion = emotions[max_index]

          cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

          shape = predictor_lm(gray_img, rect)
          shape_np = np.zeros((68, 2), dtype="int")
          for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
          shape = shape_np

        # Display the landmarks
          for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
            cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)

          resized_img = cv2.resize(test_img, (400, 400))

          cv2_imshow(resized_img)
        
        #count += 1
        #if count > 5:
        #  count = 0
        #  break
print(table)
prec_class = [0]*7
recl_class = [0]*7
for emo in range(7):
  trueP = table[emo,emo]
  falseN = -trueP
  for emo_i in range(7):
    falseN += table[emo_i,emo]
  falseP = -trueP
  for emo_i in range(7):
    falseP += table[emo,emo_i]
  print(trueP)
  prec_class[emo] = (trueP/(trueP+falseP))
  recl_class[emo] = (trueP/(trueP+falseN))
print(prec_class)
print(recl_class)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

label_list = emotions
num_list1 = prec_class
num_list2 = recl_class
x = range(7)

rects1 = plt.bar(range(7), height=num_list1, width=0.4, alpha=0.8, color='red')
rects2 = plt.bar([i + 0.4 for i in x], height=num_list2, width=0.4, color='green')
plt.ylim(0, 1)    
plt.ylabel("Performance")
plt.legend()  
plt.xticks([index + 0.2 for index in x], label_list)
plt.xlabel("Emotion")
plt.title("Performance Table")
for rect in rects1:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2, height, str(round(height,2)), ha="center", va="bottom", label="precision")
for rect in rects2:
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2, height, str(round(height,2)), ha="center", va="bottom", label="recall")
plt.show()

In [ ]:
for emo in range(7):
  #print("precision for "+op[emo]+" is: "+ str(TP_class[emo]/(TP_class[emo]+FP_class[emo])))
  print(str(TP_class[emo]) +" true positive is identified in "+str(TP_class[emo]+FP_class[emo])+" Predicted Positives")
  #print("Recall for "+op[emo]+" is: " +str(TP_class[emo]/(TP_class[emo]+FN_class[emo])))
  print(str(TP_class[emo]) +" true positive is identified in "+str(TP_class[emo]+FN_class[emo])+" Actual Positives")


In [ ]:
!unzip test_hreso.zip

In [ ]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))
def draw_eye(p1,p2,input_img):
  center_coordinates = (int((p1[0]+p2[0])/2), int((p1[1]+p2[1])/2))
  
  axesLength = (int(cv2.norm(p1,p2)*1.2), int(cv2.norm(p1,p2)*0.6))
  
  angle = angle_between(p1,p2)
  
  startAngle = 0
  
  endAngle = 360
  color = (255, 255, 255)
  thickness = -1

  input_img = cv2.ellipse(input_img, center_coordinates, axesLength, angle, startAngle, endAngle, color, thickness)

In [ ]:
model = load_model("best_model.h5")


def eye_cut(path):
  detector_lm = dlib.get_frontal_face_detector()
  predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

  test_img = cv2.imread(path)	
  gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
  cv2_imshow(test_img)

  faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)

  rects = detector_lm(gray_img, 1)
  for rect in rects:
    test_img2 = test_img.copy()
    test_img3 = test_img.copy()
    x = rect.tl_corner().x
    y = rect.tl_corner().y
    w = rect.width()
    h = rect.height()
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
    roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
    roi_gray = cv2.resize(roi_gray, (224, 224))
    img_pixels = img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis=0)
    img_pixels /= 255

    predictions = model.predict(img_pixels)

      # find max indexed array
    max_index = np.argmax(predictions[0])

    emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    predicted_emotion = emotions[max_index]

    cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    shape = predictor_lm(gray_img, rect)
    shape_np = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    shape = shape_np

        # Display the landmarks
    for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
        cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)
    eyel_l = shape[36]
    eyel_r = shape[39]
    eyer_l = shape[42]
    eyer_r = shape[45]
    mask_le = np.zeros(test_img2.shape, dtype=np.uint8)
    mask_re = np.zeros(test_img2.shape, dtype=np.uint8)
    draw_eye(eyel_l,eyel_r,mask_le)
    draw_eye(eyer_l,eyer_r,mask_re)
# Displaying the image
    eyel_l1 = eyel_l
    eyel_r1 = eyel_r
    eyer_l1 = eyer_l
    eyer_r1 = eyer_r
    masked_image = cv2.bitwise_and(test_img2,mask_le)
    cv2_imshow(masked_image)
    masked_image2 = cv2.bitwise_and(test_img2,mask_re)
    cv2_imshow(masked_image2)  

    dst = cv2.addWeighted(masked_image, 0.5, masked_image2, 0.5, 0)
    cv2_imshow(dst)  

  #resized_img = cv2.resize(test_img, (400, 400))

  #cv2_imshow(resized_img)
  return (eyel_l1,eyel_r1,eyer_l1,eyer_r1,masked_image,masked_image2)


In [ ]:
model = load_model("best_model.h5")

#path = "/content/test_hreso/happy/images (8).jpg"

detector_lm = dlib.get_frontal_face_detector()
predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')
def eye_replace(path1,path2):
  test_img = cv2.imread(path1)	
  gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
  cv2_imshow(test_img)

  faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


  eyel_l1,eyel_r1,eyer_l1,eyer_r1,masked_image,masked_image2 = eye_cut(path2)

  rects = detector_lm(gray_img, 1)
  for rect in rects:
    test_img2 = test_img.copy()
    x = rect.tl_corner().x
    y = rect.tl_corner().y
    w = rect.width()
    h = rect.height()
    cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
    roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
    roi_gray = cv2.resize(roi_gray, (224, 224))
    img_pixels = img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis=0)
    img_pixels /= 255

    predictions = model.predict(img_pixels)

      # find max indexed array
    max_index = np.argmax(predictions[0])

    emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    predicted_emotion = emotions[max_index]

    cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


    shape = predictor_lm(gray_img, rect)
    shape_np = np.zeros((68, 2), dtype="int")
    for i in range(0, 68):
        shape_np[i] = (shape.part(i).x, shape.part(i).y)
    shape = shape_np

        # Display the landmarks
    for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
        cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)
    eyel_l = shape[36]
    eyel_r = shape[39]
    eyer_l = shape[42]
    eyer_r = shape[45]

    el_scaling = (cv2.norm(eyel_l,eyel_r) / cv2.norm(eyel_l1,eyel_r1))
    print(el_scaling)

    masked_imagel = masked_image.copy() 
    print(masked_image.shape[1]/test_img.shape[1])
    print(eyel_l[0]/test_img.shape[1])
    print(eyel_l1[0]/masked_image.shape[1])
    print(eyel_r[0]/test_img.shape[1])
    print(eyel_r1[0]/masked_image.shape[1])

    width = int(masked_imagel.shape[1] * el_scaling)
    height = int(masked_imagel.shape[0] * el_scaling)
    dim = (width, height)
  
# resize image
    masked_imagel = cv2.resize(masked_imagel, dim, interpolation = cv2.INTER_AREA)
 
    mat_notrans=np.float32([[1,0,(eyel_l[0]-eyel_l1[0]*(el_scaling)+eyel_r[0]-eyel_r1[0]*(el_scaling))/2],[0,1,(eyel_l[1]-eyel_l1[1]*(el_scaling)+eyel_r[1]-eyel_r1[1]*(el_scaling))/2]])
     
    masked_imagel=cv2.warpAffine(masked_imagel,mat_notrans,(test_img.shape[1],test_img.shape[0]))

    print(masked_imagel.shape)
    cv2_imshow(masked_imagel)

    gray_eyel = cv2.cvtColor(masked_imagel, cv2.COLOR_BGR2GRAY)
    # make mask by thresholding sky image, antialias, convert to float in range 0 to 1 and make 3 channels
    mask_eyel = cv2.threshold(gray_eyel, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    mask_eyel = cv2.GaussianBlur(mask_eyel, (0,0), 2, 2)
    mask_eyel[mask_eyel<128] = 0
    mask_eyel = mask_eyel.astype(np.float32)/255
    mask_eyel = cv2.merge([mask_eyel,mask_eyel,mask_eyel])

# blend and convert back to 8-bit result
    result = test_img2 * (1 - mask_eyel) + masked_imagel * mask_eyel
    result = result.clip(0,255).astype(np.uint8)  
    #cv2_imshow(result)



    er_scaling = (cv2.norm(eyer_l,eyer_r) / cv2.norm(eyer_l1,eyer_r1))
    print(er_scaling)

    masked_imager = masked_image2.copy() 
    print(masked_image2.shape[1]/test_img.shape[1])
    print(eyer_l[0]/test_img.shape[1])
    print(eyer_l1[0]/masked_image2.shape[1])
    print(eyer_r[0]/test_img.shape[1])
    print(eyer_r1[0]/masked_image2.shape[1])

    width = int(masked_imager.shape[1] * er_scaling)
    height = int(masked_imager.shape[0] * er_scaling)
    dim = (width, height)
  
# resize image
    masked_imager = cv2.resize(masked_imager, dim, interpolation = cv2.INTER_AREA)
 
    mat_notrans=np.float32([[1,0,(eyer_l[0]-eyer_l1[0]*(er_scaling)+eyer_r[0]-eyer_r1[0]*(er_scaling))/2],[0,1,(eyer_l[1]-eyer_l1[1]*(er_scaling)+eyer_r[1]-eyer_r1[1]*(er_scaling))/2]])
     
    masked_imager=cv2.warpAffine(masked_imager,mat_notrans,(test_img.shape[1],test_img.shape[0]))

    print(masked_imager.shape)
    cv2_imshow(masked_imager)

    gray_eyer = cv2.cvtColor(masked_imager, cv2.COLOR_BGR2GRAY)
    # make mask by thresholding sky image, antialias, convert to float in range 0 to 1 and make 3 channels
    mask_eyer = cv2.threshold(gray_eyer, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    mask_eyer = cv2.GaussianBlur(mask_eyer, (0,0), 2, 2)
    mask_eyer[mask_eyer<128] = 0
    mask_eyer = mask_eyer.astype(np.float32)/255
    mask_eyer = cv2.merge([mask_eyer,mask_eyer,mask_eyer])

# blend and convert back to 8-bit result
    result = result * (1 - mask_eyer*0.9) + masked_imager * mask_eyer *0.9
    result = result.clip(0,255).astype(np.uint8)  
    cv2_imshow(result)
    return result


In [ ]:
import os.path
from os import path
if path.exists('/content/test_edited') == False:
  os.mkdir('/content/test_edited')
for i in range(7):

  if path.exists('/content/test_edited/'+op[i]) == False:

    os.mkdir('/content/test_edited/'+op[i])

  folder_dir = "/content/test_hreso/" + op[i] + "/"
  out_dir = "/content/test_edited/" + op[i] + "/"

  print("looking for eyes in folder"+folder_dir)
  for images_e in os.listdir(folder_dir):
    print("looking for eyes in "+images_e)
    # check if the image ends with png
    if (images_e.endswith(".jpg")):
      for i1 in range(3):
        i2 = random.randrange(7)
        folder_dir2 = "/content/test_hreso/" + op[i2] + "/"
        print("looking for heads in folder"+folder_dir2)
        try:
      
          images2 = random.choice(os.listdir(folder_dir2))
          new_img = eye_replace(folder_dir2 + images2, folder_dir+images_e)
          if (type(new_img) is np.ndarray): 
            cv2.imwrite(out_dir+images_e+images2, new_img)
        except:
          print("An exception occurred")
         


In [ ]:
eye_replace("/content/test_hreso/angry/download (3).jpg", "/content/test_hreso/disgust/download (3).jpg")

In [ ]:
op

In [ ]:
# import the modules
import os
from os import listdir
 
# get the path/directory
count = 0

total = 0

correct = 0
TP_class = [0]*7
FP_class = [0]*7
FN_class = [0]*7
detector_lm = dlib.get_frontal_face_detector()
predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

table = np.arange(49).reshape((7,7))
for emo in range(7):
  folder_dir = "/content/test_edited/" + op[emo] + "/"
  print(folder_dir)
  for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".jpg")):
        test_img = cv2.imread(folder_dir+images)
        #cv2_imshow(img)  
        gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
        #cv2_imshow(gray_img)    
        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        #if len(faces_detected) == 0:
        #cv2_imshow(gray_img)

        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        rects = detector_lm(gray_img, 1)
        if len(rects) == 0:
          continue
        for rect in rects:
          x = rect.tl_corner().x
          y = rect.tl_corner().y
          w = rect.width()
          h = rect.height()
          cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
          roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
          if roi_gray.size == 0:
            continue
          roi_gray = cv2.resize(roi_gray, (224, 224))
          img_pixels = img_to_array(roi_gray)
          img_pixels = np.expand_dims(img_pixels, axis=0)
          img_pixels /= 255

          predictions = model.predict(img_pixels)

      # find max indexed array
          max_index = np.argmax(predictions[0])
          total += 1
          table[emo,max_index]+=1
          if max_index == emo:

            correct += 1
            TP_class[emo] += 1
          else:
            FP_class[max_index] += 1
            FN_class[emo] += 1
          emotions = ('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')
          predicted_emotion = emotions[max_index]

          cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

          shape = predictor_lm(gray_img, rect)
          shape_np = np.zeros((68, 2), dtype="int")
          for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
          shape = shape_np

        # Display the landmarks
          for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
            cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)

          resized_img = cv2.resize(test_img, (400, 400))

          cv2_imshow(resized_img)
        
        #count += 1
        #if count > 5:
        #  count = 0
        #  break
print(table)
prec_class = [0]*7
recl_class = [0]*7
for emo in range(7):
  trueP = table[emo,emo]
  falseN = -trueP
  for emo_i in range(7):
    falseN += table[emo_i,emo]
  falseP = -trueP
  for emo_i in range(7):
    falseP += table[emo,emo_i]
  print(trueP)
  prec_class[emo] = (trueP/(trueP+falseP))
  recl_class[emo] = (trueP/(trueP+falseN))
print(prec_class)
print(recl_class)


In [ ]:
# import the modules
import os
from os import listdir
 
# get the path/directory
count = 0

total = 0

correct = 0
TP_class = [0]*7
FP_class = [0]*7
FN_class = [0]*7
detector_lm = dlib.get_frontal_face_detector()
predictor_lm = dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')

table = np.arange(49).reshape((7,7))
for emo in range(7):
  folder_dir = "/content/test_s/"
  print(folder_dir)
  for images in os.listdir(folder_dir):
 
    # check if the image ends with png
    if (images.endswith(".png")):
        test_img = cv2.imread(folder_dir+images)
        #cv2_imshow(img)  
        gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
        #cv2_imshow(gray_img)    
        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        #if len(faces_detected) == 0:
        #cv2_imshow(gray_img)

        #faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
        rects = detector_lm(gray_img, 1)
        if len(rects) == 0:
          continue
        for rect in rects:
          x = rect.tl_corner().x
          y = rect.tl_corner().y
          w = rect.width()
          h = rect.height()
          cv2.rectangle(test_img, (x, y), (x + w, y + h), (255, 0, 0), thickness=3)
          roi_gray = gray_img[y:y + w, x:x + h]  # cropping region of interest i.e. face area from  image
          if roi_gray.size == 0:
            continue
          roi_gray = cv2.resize(roi_gray, (224, 224))
          img_pixels = img_to_array(roi_gray)
          img_pixels = np.expand_dims(img_pixels, axis=0)
          img_pixels /= 255

          predictions = model.predict(img_pixels)

      # find max indexed array
          max_index = np.argmax(predictions[0])
          total += 1
          table[emo,max_index]+=1
          if max_index == emo:

            correct += 1
            TP_class[emo] += 1
          else:
            FP_class[max_index] += 1
            FN_class[emo] += 1
          emotions = ('angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise')
          predicted_emotion = emotions[max_index]

          cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

          shape = predictor_lm(gray_img, rect)
          shape_np = np.zeros((68, 2), dtype="int")
          for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
          shape = shape_np

        # Display the landmarks
          for i, (x, y) in enumerate(shape):
	    # Draw the circle to mark the keypoint 
            cv2.circle(test_img, (x, y), 1, (0, 0, 255), -1)

          resized_img = cv2.resize(test_img, (400, 400))

          cv2_imshow(resized_img)
        
        #count += 1
        #if count > 5:
        #  count = 0
        #  break
print(table)
prec_class = [0]*7
recl_class = [0]*7
for emo in range(7):
  trueP = table[emo,emo]
  falseN = -trueP
  for emo_i in range(7):
    falseN += table[emo_i,emo]
  falseP = -trueP
  for emo_i in range(7):
    falseP += table[emo,emo_i]
  print(trueP)
  prec_class[emo] = (trueP/(trueP+falseP))
  recl_class[emo] = (trueP/(trueP+falseN))
print(prec_class)
print(recl_class)
